# Wrangling behavorial data generated by OpenSesame


## Introduction
You build the experiment and ran your first participant. Now, it is time to take a look at the data you have collected.

OpenSesame outputs a *comma-separated values (csv)* file. This is a very widely used format, and you can painlessly import this file type in Python using the datafile package **pandas**. Let's import a datafile from two participants and merge those in one file:



In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# disable chained assignments
pd.options.mode.chained_assignment = None

subj1 = pd.read_csv("data/subject-3.csv", sep=",")
subj2 = pd.read_csv("data/subject-4.csv", sep=",")

df = pd.concat([subj1, subj2], ignore_index=True)

That's a lot of columns. In your "logger" file in OpenSesame, the recommended thing to do is to check the box of "Log all variables". This is the safest option, because it's easy to remove columns and you would rather not have that you missed an essential variable after doing your experiment. Let's pick the columns that we need:


In [108]:
include_columns = ['subject_nr', 'block', 'session', 'congruency_transition_type', 'congruency_type',
                   'correct', 'response_time', 'task_transition_type', 'task_type', 'cue_color']

df_trim = df[include_columns]
df_trim

,subject_nr,block,session,congruency_transition_type,congruency_type,correct,response_time,task_transition_type,task_type,cue_color
0,3,1,lowswitch,NaN,incongruent,0,1482.9738,NaN,parity,#7BB2D0
1,3,1,lowswitch,congruency-switch,congruent,1,706.7057,task-switch,magnitude,#A6A0C6
2,3,1,lowswitch,congruency-repetition,congruent,1,855.6105,task-switch,parity,#7BB2D0
3,3,1,lowswitch,congruency-switch,incongruent,1,867.8947,task-repetition,parity,#CDA277
4,3,1,lowswitch,congruency-repetition,incongruent,1,1078.9412,task-switch,magnitude,#A6A0C6
...,...,...,...,...,...,...,...,...,...,...
947,4,12,highswitch,congruency-repetition,congruent,1,699.0107,task-repetition,magnitude,#CDA277
948,4,12,highswitch,congruency-repetition,congruent,1,582.9837,task-repetition,magnitude,#8FBF35
949,4,12,highswitch,congruency-repetition,congruent,1,731.3586,task-switch,parity,#7BB2D0
950,4,12,highswitch,congruency-repetition,congruent,1,437.6794,task-repetition,parity,#A6A0C6


Before we make any changes to the dataframe, we must first be sure that all the columns are in the right [type](https://pbpython.com/pandas_dtypes.html). If we print the data types of each column, we can see that subject_nr is an integer. However, we don't intend for the dataframe to interpret "3" and "4" as numbers, since it's simply a categorization. Let's change that:

In [144]:
print(df_trim.dtypes)

df_trim['subject_nr'] = df_trim['subject_nr'].astype('category')
df_trim['correct'] = df_trim['correct'].astype('category')

print(df_trim.dtypes)

subject_nr                    category
block                            int64
session                         object
congruency_transition_type      object
congruency_type                 object
correct                       category
response_time                  float64
task_transition_type            object
task_type                       object
cue_color                       object
dtype: object
subject_nr                    category
block                            int64
session                         object
congruency_transition_type      object
congruency_type                 object
correct                       category
response_time                  float64
task_transition_type            object
task_type                       object
cue_color                       object
dtype: object


Alright, it's getting a bit more uncluttered now. The task-design is so that the last two blocks are different kind of blocks. We don't have to go in details now, but for further analysis we will have to create a dataframe without block 11 and 12. There are [many ways to conditional selection of rows](https://www.geeksforgeeks.org/selecting-rows-in-pandas-dataframe-based-on-conditions/), but here we opt to use the information that we need all blocks with a value smaller than 11.

In [109]:
# Here the last blocks should be 12
print(df_trim.tail(5))

# Conditionally select rows based on if the value in the "block" column is lower than 11
df_trim_blocks = df_trim[df_trim['block'] < 11]

# Check to see if the last block is now 10 instead of 12
print(df_trim_blocks.tail(5))

     subject_nr  block     session congruency_transition_type congruency_type  \
947           4     12  highswitch      congruency-repetition       congruent   
948           4     12  highswitch      congruency-repetition       congruent   
949           4     12  highswitch      congruency-repetition       congruent   
950           4     12  highswitch      congruency-repetition       congruent   
951           4     12  highswitch      congruency-repetition       congruent   

     correct  response_time task_transition_type  task_type cue_color  
947        1       699.0107      task-repetition  magnitude   #CDA277  
948        1       582.9837      task-repetition  magnitude   #8FBF35  
949        1       731.3586          task-switch     parity   #7BB2D0  
950        1       437.6794      task-repetition     parity   #A6A0C6  
951        0      1006.4980          task-switch  magnitude   #8FBF35  
     subject_nr  block     session congruency_transition_type congruency_type  \


Lastly, it's a bit confusing that we have only two subjects, but they are called number 3 and 4, instead of 1 and 2. Let's fix that by replacing subject 3 with subject 1, and subject 4 with subject 2. We can use the replace function of Pandas to achieve this. Then, with the pandas *unique* function we can verify that the subject numbers have been changed.

In [126]:
df_trim_blocks['subject_nr'] = df_trim_blocks['subject_nr'].replace(3, 1)
df_trim_blocks['subject_nr'] = df_trim_blocks['subject_nr'].replace(4, 2)
df_trim_blocks['subject_nr'].unique()

array([1, 2], dtype=int64)

Let's see what kind of data we are dealing with. The "session" column says lowswitch for subject 1, and highswitch for subject 2. This means that we should see less task-switch trials for subject 1 than subject 2. To check this, we can use the [pivot table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) function from pandas. Let's check:

In [127]:
piv_task_transition_exp = df_trim_blocks.pivot_table(
    index=['session'],
    columns='task_transition_type',
    aggfunc='size') # Function to aggregate columns on, here we specify "size"

piv_task_transition_exp

task_transition_type,task-repetition,task-switch
session,,
highswitch,82,248
lowswitch,245,85


It is always good practice to check if the trials count is what you expected. This experiment had a quite complex counterbalancing structure, since the researchers had to counterbalance:
- Amount of congruent/incongruent trials
- Amount of parity/magnitude trials
- Amount of congruent-switch/congruent-repetition trials

All whilst keeping the task-repetition/task-switch rate to 25/75 or 75/25 (depending on the session).
This all whilst keeping into account that the first trial of each block does not count as either repetition or switch trial. Let's see if we can use a bit more complex pivot table to get a clear picture if all of this worked out

In [128]:
piv_cong = df_trim_blocks.pivot_table(
    index=['subject_nr', 'block'],
    columns=['congruency_type'],
    aggfunc='size') # Function to aggregate columns on, here we specify "size"

piv_cong_transition = df_trim_blocks.pivot_table(
    index=['subject_nr', 'block'],
    columns=['congruency_transition_type'],
    aggfunc='size') # Function to aggregate columns on, here we specify "size"

piv_task = df_trim_blocks.pivot_table(
    index=['subject_nr', 'block'],
    columns=['task_type'],
    aggfunc='size') # Function to aggregate columns on, here we specify "size"

piv_task_transition = df_trim_blocks.pivot_table(
    index=['subject_nr', 'block'],
    columns='task_transition_type',
    aggfunc='size') # Function to aggregate columns on, here we specify "size"

# Add all dataframes to a list
dfs = [piv_cong, piv_cong_transition, piv_task, piv_task_transition]

# Merge the dataframes
pd.concat(dfs, axis=1)

congruent  incongruent  congruency-repetition  \
subject_nr block                                                  
1          1             16           18                     16   
           2             17           17                     17   
           3             17           17                     16   
           4             17           17                     17   
           5             17           17                     17   
           6             18           16                     16   
           7             17           17                     17   
           8             16           18                     17   
           9             16           18                     16   
           10            16           18                     17   
2          1             17           17                     16   
           2             17           17                     17   
           3             18           16                     16   
           4             17           17                     16   
           5             17           17                     16   
           6             16           18                     17   
           7             18           16                     16   
           8             17           17                     16   
           9             17           17                     17   
           10            18           16                     16   

                  congruency-switch  magnitude  parity  task-repetition  \
subject_nr block                                                          
1          1                     17         16      18               24   
           2                     16         18      16               24   
           3                     17         17      17               25   
           4                     16         18      16               25   
           5                     16         17      17               24   
           6                     17         17      17               24   
           7                     16         18      16               25   
           8                     16         16      18               24   
           9                     17         18      16               25   
           10                    16         17      17               25   
2          1                     17         16      18                9   
           2                     16         17      17                9   
           3                     17         17      17                8   
           4                     17         18      16                8   
           5                     17         17      17                8   
           6                     16         17      17                8   
           7                     17         16      18                8   
           8                     17         16      18                8   
           9                     16         18      16                8   
           10                    17         16      18                8   

                  task-switch  
subject_nr block               
1          1                9  
           2                9  
           3                8  
           4                8  
           5                9  
           6                9  
           7                8  
           8                9  
           9                8  
           10               8  
2          1               24  
           2               24  
           3               25  
           4               25  
           5               25  
           6               25  
           7               25  
           8               25  
           9               25  
           10              25

We are repeating quite a lot of code. Whenever you notice that happen, you can probably shorten the code. Let's try it out:

In [129]:
columns_to_check = ['task_type', 'congruency_type',
                    'task_transition_type', 'congruency_transition_type']

dfs = []
for column in columns_to_check:
    piv = df_trim_blocks.pivot_table(
        index=['subject_nr', 'block'],
        columns=[column],
        aggfunc='size') # Function to aggregate columns on, here we specify "size"

    dfs.append(piv)

# Merge the dataframes
pd.concat(dfs, axis=1)

magnitude  parity  congruent  incongruent  task-repetition  \
subject_nr block                                                               
1          1             16      18         16           18               24   
           2             18      16         17           17               24   
           3             17      17         17           17               25   
           4             18      16         17           17               25   
           5             17      17         17           17               24   
           6             17      17         18           16               24   
           7             18      16         17           17               25   
           8             16      18         16           18               24   
           9             18      16         16           18               25   
           10            17      17         16           18               25   
2          1             16      18         17           17                9   
           2             17      17         17           17                9   
           3             17      17         18           16                8   
           4             18      16         17           17                8   
           5             17      17         17           17                8   
           6             17      17         16           18                8   
           7             16      18         18           16                8   
           8             16      18         17           17                8   
           9             18      16         17           17                8   
           10            16      18         18           16                8   

                  task-switch  congruency-repetition  congruency-switch  
subject_nr block                                                         
1          1                9                     16                 17  
           2                9                     17                 16  
           3                8                     16                 17  
           4                8                     17                 16  
           5                9                     17                 16  
           6                9                     16                 17  
           7                8                     17                 16  
           8                9                     17                 16  
           9                8                     16                 17  
           10               8                     17                 16  
2          1               24                     16                 17  
           2               24                     17                 16  
           3               25                     16                 17  
           4               25                     16                 17  
           5               25                     16                 17  
           6               25                     17                 16  
           7               25                     16                 17  
           8               25                     16                 17  
           9               25                     17                 16  
           10              25                     16                 17

Nice. It's cleaner and also way easier to change this code if you want to check another column for example. The counterbalancing looks good on a block level. Let's do some first checks on whether the results are what we expect. Let's first remove all the incorrect trials, we aren't interested in those at the moment.

In [130]:
df_correct = df_trim_blocks[df_trim_blocks['correct'] == 1]
df_correct

,subject_nr,block,session,congruency_transition_type,congruency_type,correct,response_time,task_transition_type,task_type,cue_color
1,1,1,lowswitch,congruency-switch,congruent,1,706.7057,task-switch,magnitude,#A6A0C6
2,1,1,lowswitch,congruency-repetition,congruent,1,855.6105,task-switch,parity,#7BB2D0
3,1,1,lowswitch,congruency-switch,incongruent,1,867.8947,task-repetition,parity,#CDA277
4,1,1,lowswitch,congruency-repetition,incongruent,1,1078.9412,task-switch,magnitude,#A6A0C6
5,1,1,lowswitch,congruency-repetition,incongruent,1,819.4645,task-repetition,magnitude,#8FBF35
...,...,...,...,...,...,...,...,...,...,...
810,2,10,highswitch,congruency-repetition,congruent,1,321.1855,task-switch,parity,#7BB2D0
811,2,10,highswitch,congruency-switch,incongruent,1,844.0940,task-switch,magnitude,#8FBF35
812,2,10,highswitch,congruency-repetition,incongruent,1,674.5391,task-switch,parity,#A6A0C6
814,2,10,highswitch,congruency-switch,congruent,1,1353.6886,task-switch,magnitude,#CDA277


A really convenient way to quickly get an idea of your numerical columns, you can use the describe function of pandas:

In [131]:
df_correct['response_time'].describe()

count     599.000000
mean      725.344337
std       233.487524
min       124.173900
25%       550.588900
50%       655.866000
75%       876.164300
max      1443.708500
Name: response_time, dtype: float64

  We expect people to be slower when they have to switch from task, in comparison to when they can do the same task as on the previous trial. This is what we call **switch cost**. To show this in a table-format, we can again use the pivot_table function from pandas.

In [135]:
#check counts
switch_table = pd.pivot_table(
    df_correct,
    values="response_time", # The value that will be summarized
    index=["subject_nr"], # The rows to summarize over
    columns=["task_transition_type"], # The columns to summarize over
    aggfunc=np.mean, # Calculate the mean response time per subject per task type
)

switch_table

task_transition_type,task-repetition,task-switch
subject_nr,,
1,749.989603,888.007569
2,638.137917,675.488397


It is pretty clear that incongruent trials are slower than congruent trials, but we can make it even clearer by showing the difference between the two columns. We can make a new column, and input in that column the difference between the incongruent and congruent columns:

In [136]:
switch_table['switch cost'] = switch_table['task-switch'] - switch_table['task-repetition']
switch_table

task_transition_type,task-repetition,task-switch,switch cost
subject_nr,,,
1,749.989603,888.007569,138.017966
2,638.137917,675.488397,37.350480


Lastly, you can use the handy function *describe* to get a quick peek at the response times.

In [75]:
df_correct['response_time'].describe()

count     638.000000
mean      724.993290
std       221.994389
min       387.936300
25%       555.015275
50%       651.329350
75%       869.120000
max      1469.761100
Name: response_time, dtype: float64

What we've done here is just a subset of the myriad of possibilities how you can change your dataframe in a way that is tidy and gives you a better overview of what you're dealing with. [This cheat sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) gives a nice overview of the things we discussed and more! Use the cheat sheet for the following exercises.

> **Note:** The developers of OpenSesame have also created a Python package for working with column-based and continuous data, called [DataMatrix](http://pydatamatrix.eu/0.15/index/). It's similar to the Pandas package we've been working with in this tutorial, but has some crucial differences in syntax. We have opted for the more versatile and widely used Pandas package, but be aware that the OpenSesame website and tutorials can sometimes refer to DataMatrixes instead of DataFrames.

### Exercise 1
We've removed incorrect trials from our experiment, and looked at the switch cost effect after. However, we should be aware that there is a difference in how many incorrect trials there are per condition. Show the amount of correct/incorrect trials per congruency using a pivot table.

In [ ]:
# Pivot table

### Exercise 2
Next to removing incorrect trials from a dataframe, an often done dataframe manipulation is to remove outliers from your dataframe. These can be trials where participants where unrealistically quick, or just too slow to test the effect you are interested in.

From the dataframe with correct trials only, remove all trials that have a reaction time below 400ms and above 1000ms. Use a conditional selection of rows, and use only one line of code.

In [ ]:
# df_correct

### Exercise 3
What is a more objective way to do outlier detection? Name two outlier detection methods, one that is suited for normally distributed data, and one that is suited for non-normally distributed data.

*Answer to exercise 3*

During the next session we will dive deeper into outlier detection methods.